In [2]:
# This a simple CNN on the CIFAR10 data set, consists of 60000 32x32 colour images in 10 classes, with 6000 images per class.


# Data set is download at : https://www.cs.toronto.edu/~kriz/cifar.html


%cd '/content/drive/MyDrive/Tutorial CIFAR10/cifar-10-python'

CIFAR_DIR= 'cifar-10-batches-py/'
CIFAR_DIR

/content/drive/MyDrive/Tutorial CIFAR10/cifar-10-python


'cifar-10-batches-py/'

In [3]:
def unpickle(file):
  import pickle
  with open(file, 'rb') as fo:
    cifar_dict = pickle.load(fo, encoding='bytes')
  return cifar_dict


In [4]:
dirs=['batches.meta', 'data_batch_1', 'data_batch_2', 'data_batch_3', 'data_batch_4', 'data_batch_5', 'test_batch']

In [6]:
all_data = [0,1,2,3,4,5,6]


In [7]:
for i,direc in zip (all_data, dirs):
  all_data[i] = unpickle (CIFAR_DIR+direc)


In [9]:
batch_meta = all_data[0]
data_batch1 = all_data[1]
data_batch2 = all_data[2]
data_batch3 = all_data[3]
data_batch4 = all_data[4]
data_batch5 = all_data[5]
test_batch = all_data[6]


In [10]:
batch_meta

{b'num_cases_per_batch': 10000,
 b'label_names': [b'airplane',
  b'automobile',
  b'bird',
  b'cat',
  b'deer',
  b'dog',
  b'frog',
  b'horse',
  b'ship',
  b'truck'],
 b'num_vis': 3072}

In [ ]:
#Checking training imagesm you would change the number of image in the last line and check that record.

# import matplotlib.pyplot as plt
# %matplotlib inline
# import numpy as np

# X = data_batch1[b"data"]
# X = X.reshape(10000, 3, 32, 32).transpose(0,2,3,1).astype("uint8")
# X[0].max()
# (X[0]/255).max()
# plt.imshow(X[2])


In [54]:
#Use a helper function to handle data:




def one_hot_encode(vec, vals=10):

  n = len(vec)
  out = np.zeros((n, vals))
  out [range(n), vec]= 1
  return out

class CifarHelper():
  def __init__(self):

    self.i = 0
    self.all_train_batches = [data_batch1, data_batch2, data_batch3, data_batch4, data_batch5] 
    self.test_batch = [test_batch]
    self.training_images = None 
    self.training_labels = None
    self.test_images = None 
    self.test_labels = None


  def set_up_images(self):
    print("Setting Up Training Images and Labels")
    self.training_images = np.vstack ([d[b"data"] for d in self.all_train_batches])
    train_len = len(self.training_images)
    self.training_images = self.training_images.reshape(train_len, 3,32,32). transpose(0,2,3,1)/255 
    self.training_labels = one_hot_encode(np.hstack ([d[b"labels"] for d in self.all_train_batches]), 10)
    print("Setting Up Test Images and Labels")
    self.test_images = np.vstack ([d[b"data"] for d in self.test_batch])
    test_len = len(self.test_images)
    self.test_images = self.test_images.reshape(test_len, 3, 32, 32). transpose(0,2,3,1)/255

  def next_batch(self, batch_size):
    x= self.training_images[self.i:self.i+batch_size].reshape(100,32,32,3)
    y= self.training_labels[self.i:self.i+batch_size]
    self.i=(self.i+batch_size) % len(self.training_images)
    return x, y




In [29]:
!pip install tensorflow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [55]:
ch = CifarHelper()
ch.set_up_images()
import tensorflow.compat.v1 as tf

tf.disable_v2_behavior()

x = tf.placeholder(tf.float32, shape=[None,32,32,3]) 
y_true = tf.placeholder (tf.float32, shape=[None, 10])
hold_prob= tf.placeholder(tf.float32)


Setting Up Training Images and Labels
Setting Up Test Images and Labels


In [33]:
def init_weights (shape):
  init_random_dist = tf.truncated_normal (shape, stddev=0.1) 
  return tf.Variable(init_random_dist)

def init_bias (shape):
  init_bias_vals = tf.constant(0.1, shape=shape)
  return tf.Variable(init_bias_vals)

def conv2d(x, W):
  return tf.nn.conv2d(x, W, strides = [1, 1, 1, 1], padding='SAME')

def max_pool_2by2(x):
  return tf.nn.max_pool (x, ksize=[1, 2, 2, 1],
                         strides=[1, 2, 2, 1], padding='SAME')
  
def convolutional_layer (input_x, shape):
  W = init_weights (shape)
  b = init_bias ([shape[3]])
  return tf.nn.relu (conv2d(input_x, W) + b)

def normal_full_layer (input_layer, size): 
  input_size = int(input_layer.get_shape()[1]) 
  W = init_weights ([input_size, size])
  b = init_bias ([size])
  return tf.matmul (input_layer, W) + b


In [34]:
convo_1 = convolutional_layer (x, shape=[4,4,3,32])
convo_1_pooling = max_pool_2by2(convo_1)

convo_2 = convolutional_layer (convo_1_pooling, shape=[4,4,32,64]) 
convo_2_pooling = max_pool_2by2(convo_2)


In [36]:
# Create a fully connected layer:
convo_2_flat = tf.reshape(convo_2_pooling, [-1,8*8*64])

full_layer_one = tf.nn.relu (normal_full_layer (convo_2_flat,1024))
full_one_dropout = tf.nn.dropout(full_layer_one, keep_prob=hold_prob)



Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [37]:
y_pred = normal_full_layer (full_one_dropout, 10)
y_pred

<tf.Tensor 'add_3:0' shape=(?, 10) dtype=float32>

In [38]:
#Loss Function
cross_entropy = tf.reduce_mean (tf.nn.softmax_cross_entropy_with_logits (labels=y_true, logits=y_pred))


Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [40]:
#Create the optimizer:
optimizer = tf.train. AdamOptimizer (learning_rate=0.001) 
train=optimizer.minimize (cross_entropy)

In [41]:
#Create a variable to initialize all the global variables:
init = tf.global_variables_initializer()


In [59]:
with tf.Session() as sess:
  sess.run(tf.global_variables_initializer())
  for i in range(500):
    batch=ch.next_batch(100)
    sess.run(train, feed_dict={x: batch [0], y_true: batch [1], hold_prob: 0.5})
    # PRINT OUT A MESSAGE EVERY 100 STEPS
    if i%100 == 0:
      print('Currently on step {}'.format(i))
      print('Accuracy is:')
      # Test the Train Model
      matches = tf.equal(tf.argmax(y_pred,1), tf.argmax (y_true,1))
      acc= tf.reduce_mean (tf.cast (matches, tf.float32))
      print(sess.run(acc, feed_dict={x:ch.test_images, y_true: ch.test_labels, hold_prob:1.0}))
      print('\n')


InvalidArgumentError: ignored